In [1]:
import csv
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import datetime as dt
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import os

In [3]:
!git clone https://github.com/guillemr/robust-fpop
!R CMD build robust-fpop
!R CMD INSTALL robseg_*.tar.gz


robjects.r("install.packages('changepoint', lib = '/usr/local/lib/R/site-library')")
robseg = importr('robseg')
robjects.r.source("robust-fpop/Simulation.R");

fatal: destination path 'robust-fpop' already exists and is not an empty directory.
* checking for file ‘robust-fpop/DESCRIPTION’ ... OK
* preparing ‘robseg’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘robseg_2019.07.02.tar.gz’

* installing to library ‘/Users/zhaoyuxuan/Library/R/3.5/library’
* installing *source* package ‘robseg’ ...
** libs
x86_64-apple-darwin13.4.0-clang++  -I"/Users/zhaoyuxuan/anaconda3/lib/R/include" -DNDEBUG   -D_FORTIFY_SOURCE=2 -mmacosx-version-min=10.9 -I/Users/zhaoyuxuan/anaconda3/include   -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden  -fmessage-length=0 -I/Users/zhaoyuxuan/anaconda3/include -fdebug-prefix-map==/usr/local/src/conda/- -fdebug-prefix-map==/usr/local/src/conda-prefix  -c BinSeg_MultiDim.cpp -o 

R[write to console]: Warning in install.packages("changepoint", lib = "/usr/local/lib/R/site-library") :
R[write to console]: 
 
R[write to console]:  'lib = "/usr/local/lib/R/site-library"' is not writable



Would you like to use a personal library instead? (yes/No/cancel) yes
--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Ghent) [https]
 8: Brazil (BA) [https]
 9: Brazil (PR) [https]
10: Brazil (RJ) [https]
11: Brazil (SP 1) [https]
12: Brazil (SP 2) [https]
13: Bulgaria [https]
14: China (Beijing 1) [https]
15: China (Beijing 2) [https]
16: China (Hong Kong) [https]
17: China (Guangzhou) [https]
18: China (Lanzhou) [https]
19: China (Nanjing) [https]
20: China (Shanghai 1) [https]
21: Colombia (Cali) [https]
22: Costa Rica [https]
23: Denmark [https]
24: East Asia [https]
25: Ecuador (Cuenca) [https]
26: Ecuador (Quito) [https]
27: France (Lyon 1) [https]
28: France (Lyon 2) [https]
29: France (Marseille) [https]
30: Germany (Erlangen) [https]
31: Germany (Münster) [ht

R[write to console]: trying URL 'https://mirror.marwan.ma/cran/src/contrib/changepoint_2.2.2.tar.gz'

R[write to console]: Content type 'application/octet-stream'
R[write to console]:  length 250737 bytes (244 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[w

In [4]:
def cpd_indiv(hw_type, df, lthreshold):
    
    global processed_configs_count
    
    for idx, grp in df.groupby(factor_list):
        
        # Do not consider subsets that include less than 200 measurements
        if len(grp) < 200:
            continue
            
        processed_configs_count += 1

        vals = grp[metric].values
        timestmaps = grp.timestamp.values
        vals_str = ",".join([str(el) for el in vals])
        # Create a list with these values and pass it into the R environment
        robjects.r("x.data <- c(%s)" % (vals_str))

        # Run CPD with the selected threshold
        res=robjects.r("""
            est.sd <- varDiff(x.data)
            res.ou <- fpop_intern(x.data/est.sd,  test.stat="Outlier", pen.value=2*log(length(x.data)), lthreshold=%f)
            cpt <- res.ou$cpts[-length(res.ou$cpts)]
            """ % lthreshold)

        mean_vals = robjects.r("res.ou$smt.signal * est.sd") 
        for v in robjects.r("cpt"):
            # For each changepoint, estimate the corresponding change in means (relative change, %)
            percent_change = (mean_vals[v] - mean_vals[v-1]) / mean_vals[v-1] * 100.0
            indiv_cps.loc[len(indiv_cps)] = [hw_type, "%.2f" % lthreshold] + [str(x) for x in list(idx)] + [timestmaps[v], percent_change] 

        # Estimate legths of segments, in seconds
        timestmaps_min = timestmaps.min()
        timestmaps_max = timestmaps.max()
        if len(robjects.r("cpt")) == 0:
            segment_lengths = [timestmaps_max-timestmaps_min]
        elif len(robjects.r("cpt")) == 1:
            segment_lengths = [timestmaps[robjects.r("cpt")[0]]-timestmaps_min, timestmaps_max-timestmaps[robjects.r("cpt")[0]]]
        else:
            segment_lengths = [timestmaps[robjects.r("cpt")[0]]-timestmaps_min, timestmaps_max-timestmaps[robjects.r("cpt")[-1]]]
            for cpt_idx in range(len(robjects.r("cpt")))[1:]:
                  segment_lengths.append(timestmaps[robjects.r("cpt")[cpt_idx]]-timestmaps[robjects.r("cpt")[cpt_idx-1]])
        for sl in segment_lengths:
            indiv_durations.loc[len(indiv_durations)] = [hw_type, "%.2f" % lthreshold] + [str(x) for x in list(idx)] + [sl / (3600.0 * 24)] 

In [5]:
def cpd_aggr():
    """ 
    Run cpd_indiv() a bunch of times and aggregate/save results; repeat for different CPD thresholds.
    This function will be reused for memory and disk CPD. 
    """
    global processed_configs_count
    global total_measurements

    for hw_type, hw_type_grp in df.groupby("hw_type"):

        print ("\nProcessing hw_type:", hw_type)
        total_measurements += len(hw_type_grp)

        processed_config_count = 0
        for idx, grp in hw_type_grp.groupby(factor_list):
            if len(grp) >= 200:
                processed_config_count += 1

        meta_df.loc[len(meta_df)] = [hw_type, len(hw_type_grp), processed_config_count,
                                            hw_type_grp.timestamp.min(), hw_type_grp.timestamp.max()]
         
        for lthreshold in lthreshold_list:  
      
            # Run the analysis for selected configuration
            cpd_indiv(hw_type, hw_type_grp, lthreshold)

        print ("Processed configs:", processed_configs_count)
        print ("Processed measurements:", total_measurements)

    meta_df.to_csv("%s/meta" % (dest_dir), index=False)
    indiv_durations.to_csv("%s/segment-durations" % (dest_dir), index=False)

    for lthreshold, grp in indiv_cps.groupby("lthreshold"):
        
        grp.to_csv("%s/indiv-%.2f" % (dest_dir, float(lthreshold)), index=False)

        aggr_df = pd.DataFrame()
        for hw, subgrp in grp.groupby("hw_type"):
            
            # Working only for CPU data/factor_list
            #configs_str = (subgrp["testname"] + "-" + subgrp["total_threads"] + "-" + subgrp["dvfs"]+ "-" + subgrp["socket_num"] + ";").tolist()
            # More general version: 
            configs_str = ["-".join(x.values)+";" for idx, x in subgrp[factor_list].astype(str).iterrows()]

            timeline = pd.DataFrame({"configs": configs_str,
                                    "direction": subgrp.percent_change.apply(lambda x: "U" if x > 0 else "D").tolist(),
                                    "percent_changes": subgrp.percent_change.apply(lambda x: str(x) + ";").tolist()},
                                    index=pd.to_datetime(subgrp.timestamp, unit='s'))
            t_count = timeline.resample('D').sum()
            t_count["count"] = t_count["configs"].apply(lambda x: len(str(x).split(";"))-1)
            t_count["hw_type"] = hw
            aggr_df = pd.concat([aggr_df, t_count])

        aggr_df.to_csv("%s/aggr-%.2f" % (dest_dir, float(lthreshold)))